In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [ ]:
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),
                          chrome_options=options)
infos_main_screen = []

for i in range(1, 12):
    url = f"https://polkastarter.com/projects?page={i}#search"
    driver.get(url)

    f = driver.find_element(
        By.XPATH, '//table[@id="ps--funded-projects__table"]//tbody')
    for name in f.find_elements_by_xpath(".//tr"):

        info = name.find_elements_by_xpath(".//td")
        temp_list = [i.text for i in info]
        infos_main_screen.append(temp_list)
        
driver.quit()


In [ ]:
cols = [
    "Project Name", "Type", "Participants", "Total Raised", "Current Price",
    "ATH Since IDO", "Ended in (UTC)", "Networks"
]

df = pd.DataFrame(infos_main_screen, columns=cols).drop_duplicates()

In [ ]:
df[['Project Name', 'Ticker']] = pd.Series(df['Project Name']).str.split("\n",
                                                              expand=True)

In [ ]:
df['Link'] = "https://polkastarter.com/projects/" + (
    df['Project Name'].str.lower()).str.replace(" ", "-").str.replace(
        ".", "-")


In [ ]:
df

In [ ]:
df['Link'] = df['Link'].str.replace("polychain-monsters", "polkamon")
df['Link'] = df['Link'].str.replace("shyft-network-inc-", "shyft-network-inc")


In [ ]:
df.to_excel("../raw_data/dados_pag_inicial_polka.xlsx")

In [ ]:
cont_final = []
for url in df['Link']:
    print(url)
    content = requests.get(url)
    breakpoint()
    soup = BeautifulSoup(content.content, 'html.parser')
    data = soup.find("div", {"id": "app-content"}).find_next()
    dc = json.loads(data.attrs['data-project-json'])
    cont_final.append(dc)


In [ ]:
pd.DataFrame(cont_final).drop("description", axis=1).to_excel("../raw_data/scraped_data_polka.xlsx")

In [ ]:
options = Options()
# options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),
                          chrome_options=options)

all = []
for url in df['Link']:

    driver.get(url)
    tx = driver.find_element(By.XPATH, '//*[@id="description"]/div/div[1]')
    tags = tx.find_elements(By.TAG_NAME, "h2")
    tgs = [(i.text) for i in tags]

    data = {"link":url}
    if len(tgs) == 1 :
        data[tgs[0]] = begin.replace(tgs[0], "")
    elif len(tgs) == 0:
        data["No Label"] = tx.text

    else:
        begin, end = tx.text.split(tgs[1])
        data[tgs[0]] = begin.replace(tgs[0], "")

        for iten in tgs[2:]:

            begin, end = end.split(iten)
            data[iten] = begin


    all.append(data)
driver.quit()

In [ ]:
df = pd.DataFrame(all)

In [ ]:
for col in df.columns:

    df[col] = df[col].str.replace("\n", " ").str.replace("  ", " ")

In [ ]:
df.to_excel("../raw_data/text_polka.xlsx", index=False)